In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.model_selection import cross_val_score,cross_val_predict,cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import explained_variance_score

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.models import save_model, load_model
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [ ]:
params = {'font.family'    : 'serif',  # 衬线字体族
          'font.serif'     : 'Times New Roman',
          'legend.fontsize': 20,
          'figure.dpi'     : 100,
          'savefig.dpi'    : 600,
        #   'figure.figsize' : (15, 5),
          'axes.labelsize' : 20,
          'axes.titlesize' : 20,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
          }
plt.rcParams.update(params)

In [ ]:
data = pd.read_excel("FE_trainData_AB_AB.xlsx", index_col=0, header=0)

x = data.iloc[:, :-1]
y = data.iloc[:, -1]
x_train_dim = x.shape[1]

x = MinMaxScaler().fit_transform(x)
y = np.array(y)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [ ]:
def rmse(y_true, y_pred):
	"""自定义损失函数RMSE"""
	rmse = K.sqrt(K.mean(K.square(y_pred - y_true)))
	return rmse

def r_square(y_true, y_pred):
	"""自定义评估函数R2"""
	RSS = K.sum(K.square(y_true - y_pred))
	TSS = K.sum(K.square(y_true-K.mean(y_true)))
	r_square = 1 - RSS/TSS
	return r_square

In [ ]:
model_source_domin = load_model('0_best_model.h5', custom_objects={'rmse':rmse, 'r_square':r_square})

# 模型预测
y_pred = model_source_domin.predict(x)

# 保存每一次交叉验证的预测结果
y_pred = y_pred.reshape(-1)
pred_data = pd.DataFrame({"y":y,"y_pred":y_pred})
pred_data.to_excel("pred_data.xlsx")

# 预测得分
train_R2 = r2_score(y, y_pred)
train_RMSE = np.sqrt(mean_squared_error(y, y_pred))
train_MAE = mean_absolute_error(y, y_pred)

print('R2:', train_R2, 'RMSE:', train_RMSE, 'MAE', train_MAE)

# 绘制model预测结果散点图
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 5), constrained_layout=True)
fig.patch.set_alpha(1)  # 设置绘图背景不透明
plt.scatter(y, y_pred, c='y', s=50, alpha=0.5, label='train set')
plt.plot([-4, 0.2],[-4, 0.2], c='k', linewidth=1)
plt.axis([-4, 0.2,-4, 0.2])
plt.xlabel('DFT')
plt.ylabel('ANN')
plt.legend(loc='upper left', framealpha=0, markerscale=1.5)
# 在图中绘制表格
# train_R2   = "{:.2f}".format(train_R2)
# train_RMSE = "{:.4f}".format(train_RMSE)
# train_MAE  = "{:.4f}".format(train_MAE)

# row_labels = [r'$R^2$','RMSE','MAE']
# col_labels = ['Train']
# table_vals = [[train_R2, test_R2],[train_RMSE, test_RMSE] ,[train_MAE, test_MAE]]
# table = plt.table(cellText=table_vals, 
#                 rowLabels=row_labels, 
#                 colLabels=col_labels,
#                 colWidths=[0.2]*3,
#                 cellLoc='center', 
#                 rowLoc='center', 
#                 colLoc='center',
#                 loc='lower right',
#                 )
# table.set_fontsize(16) # 表格字体大小
# table.scale(1, 1.8)    # 表格缩放
plt.savefig('result.png')



In [ ]:
for i in range(10):
    model_source_domin = load_model('{}_best_model.h5'.format(i), custom_objects={'rmse':rmse, 'r_square':r_square})

    # 模型预测
    y_pred = model_source_domin.predict(x)

    # 保存每一次交叉验证的预测结果
    y_pred = y_pred.reshape(-1)
    pred_data = pd.DataFrame({"y":y,"y_pred":y_pred})
    pred_data.to_excel("pred_data.xlsx")

    # 预测得分
    train_R2 = r2_score(y, y_pred)
    train_RMSE = np.sqrt(mean_squared_error(y, y_pred))
    train_MAE = mean_absolute_error(y, y_pred)

    print('R2:', train_R2, 'RMSE:', train_RMSE, 'MAE', train_MAE)